In [1]:
# import asyncio
# from pathlib import Path
# from autogen_ext.models.openai import OpenAIChatCompletionClient
# from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
# from autogen_agentchat.agents import AssistantAgent
# from autogen_core import CancellationToken
# from autogen_ext.models.ollama import OllamaChatCompletionClient
# from autogen_core.models import ModelInfo , UserMessage
# import os 

In [12]:
# env_vars = os.environ

In [2]:
# model_client = OpenAIChatCompletionClient(
#     model="qwen2.5:14b-instruct-q8_0",
#     base_url="http://localhost:11434/v1",
#     api_key="placeholder",
#     model_info={
#         "vision": False,
#         "function_calling": True,
#         "json_output": False,
#         "family": "unknown",
#     },
# )

In [30]:
# model_client=OllamaChatCompletionClient(
#     model="llama3.2:3b-instruct-fp16",
#     host="http://localhost:11434",
#     response_format=WriteFileArgs,
#     model_info={
#         "vision": False,
#         "function_calling": True,
#         "json_output": False,
#         "family": "unknown",
#     },
#                           )

In [326]:

# # Setup server params for local filesystem access
# desktop = str(Path.home() / "Desktop")

# server_params = StdioServerParams(
#     command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", "/oper/ch/code"]
# )

# # Get all available tools from the server
# tools = await mcp_server_tools(server_params)

# # Create an agent that can use all the tools
# agent = AssistantAgent(
#     name="file_manager",
#     model_client=model_client,
#     tools=tools,  # type: ignore
#     reflect_on_tool_use=True
# )

# # The agent can now use any of the filesystem tools
# await agent.run(task="读取test.txt文件，编辑文件删除第二行", cancellation_token=CancellationToken())
# # await model_client.create(messages=[UserMessage(source="user", content="use write_file tool to Create a file called test.txt with some content")], tools=tools)

In [9]:
# asyncio.run(main())

In [3]:
server_params = StdioServerParams(
    command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", "/home/hao/Desktop"]
)

In [4]:
tools = await mcp_server_tools(server_params)

In [10]:
from mcp import ClientSession, StdioServerParameters, types

In [3]:
server_params = StdioServerParams(
    command="mcp", args=[ "run", "/oper/ch/code/mcp_service.py"]
)

In [2]:
import os
env_vars = os.environ
env_vars["PLAYWRIGHT_WS_ENDPOINT"]="ws://localhost:5544/"
server_params = StdioServerParams(
    command="npx", args=[ "@playwright/mcp@latest"],env=env_vars
)

In [3]:
tools = await mcp_server_tools(server_params)

In [327]:
# for tool in tools:
#     # print(tool.name)
#     print(f"Tool Name: {tool.name}, Description: {tool.description}")

In [43]:
# for tool in tools:
#     if tool.name == "write_file":
#         print(f"Parameters for {tool.name}: {tool.schema}")
#         break

In [11]:
from pydantic import BaseModel

class WriteFileArgs(BaseModel):
    path: str
    content: str

In [45]:
args = WriteFileArgs(path="/home/hao/Desktop/test.txt", content="This is a test file.")

In [46]:
result = await tool.run(args=args, cancellation_token=CancellationToken())

In [47]:
# for tool in tools:
#     # print(tool.name)
#     print(f"Tool Name: {tool.name}, Description: {tool.description}")

In [38]:
await main()

In [39]:
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])
tools = await mcp_server_tools(fetch_mcp_server)

In [46]:
result = await model_client.create(messages=[UserMessage(source="user", content="Summarize the content of https://newsletter.victordibia.com/p/you-have-ai-fatigue-thats-why-you")], tools=tools)
print(result)

finish_reason='function_calls' content=[FunctionCall(id='call_mkchfxlp', arguments='{"max_length":2000,"url":"https://newsletter.victordibia.com/p/you-have-ai-fatigue-thats-why-you"}', name='fetch')] usage=RequestUsage(prompt_tokens=309, completion_tokens=119) cached=False logprobs=None thought=None


In [47]:
result

CreateResult(finish_reason='function_calls', content=[FunctionCall(id='call_mkchfxlp', arguments='{"max_length":2000,"url":"https://newsletter.victordibia.com/p/you-have-ai-fatigue-thats-why-you"}', name='fetch')], usage=RequestUsage(prompt_tokens=309, completion_tokens=119), cached=False, logprobs=None, thought=None)

In [15]:
from autogen_ext.tools.mcp import SseMcpToolAdapter, SseServerParams

In [26]:
server_params = SseServerParams(
    url="http://127.0.0.1:8800/sse",  # Adjust the URL to match your FastMCP server
    headers={"Content-Type": "application/json"}
    # api_key="your_api_key_if_needed"  # Optional, if your server requires authentication
)

In [27]:
adapter = await SseMcpToolAdapter.from_server_params(server_params,tool_name='echo_tool')

In [32]:
adapter.schema

{'name': 'echo_tool',
 'description': 'Echo a message as a tool',
 'parameters': {'type': 'object',
  'properties': {'message': {'title': 'Message', 'type': 'string'}},
  'required': ['message'],
  'additionalProperties': False},
 'strict': False}

In [36]:
import asyncio
from mcp.client.sse import sse_client
from mcp.client.session import ClientSession

async def get_tools():
    async with sse_client("http://localhost:8800/sse") as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            # List available tools
            return await session.list_tools()
async def main():
    tools = await get_tools()
    print(tools.tools)

In [40]:
await main()

[Tool(name='echo_tool', description='Echo a message as a tool', inputSchema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'echo_toolArguments', 'type': 'object'})]


In [11]:
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from contextlib import AsyncExitStack
import asyncio
# import nest_asyncio
# Create server parameters for stdio connection
# server_params = StdioServerParameters(
#     command="python",  # Executable
#     args=["example_server.py"],  # Optional command line arguments
#     env=None,  # Optional environment variables
# )
# server_params = StdioServerParameters(
#     command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", "/oper/ch/code"]
# )

In [2]:
# async with stdio_client(server_params) as (read, write):
#     async with AsyncExitStack() as stack:
#         session = await stack.enter_async_context(ClientSession(read, write))

#         await session.initialize()

#         # 获取工具列表
#         response = await session.list_tools()
#         tools = response.tools

#         print("连接成功，以下是可用工具：")
#         # for tool in tools:
#         #     print(f"- {tool.name}: {tool.description}")
#         tool_descriptions = "\n".join(
#             f"{tool.name}: {tool.description}" for tool in tools
#         )
#         content=await session.call_tool('read_file',{'path':'/oper/ch/code/test.txt'})

连接成功，以下是可用工具：


In [3]:
# print(content.content[0].text)

国破山河在，城春草木深。
感时花溅泪，恨别鸟惊心。
烽火连三月，家书抵万金。
白头搔更短，浑欲不胜簪。


In [83]:
# print(await session.call_tool('read_file',{'path':'/oper/ch/code/test.txt'}))

In [4]:
# sessions = await asyncio.gather(*(mcp_client(p) for p in server_params_list))

In [9]:
# response=await sessions[0]['session'].list_tools()

In [17]:
# for line in response.tools:
#     print(line.name,line.description)

In [12]:
async def mcp_client(server_params):
    client = stdio_client(server_params)
    read, write = await client.__aenter__()

    session = ClientSession(read, write)
    await session.__aenter__()
    await session.initialize()

    return {"session": session, "read": read, "write": write, "client": client}
# # 关闭单个
# async def close_mcp_session(session, read, write, client):
#     # await session.__aexit__(None, None, None)
#     # await client.__aexit__(None, None, None)
#     # 如果你直接持有 read/write，也可以单独关闭它们：
#     await read.aclose()
#     await write.aclose()

# async def get_tool_description(server_params_list):
#     try:
#         sessions = await asyncio.gather(*(mcp_client(p) for p in server_params_list))
#         tool_descriptions=''
#         session_index={}
#         for idx,tool in enumerate(sessions):
#             response=await tool['session'].list_tools()
#             descriptions = "\n".join(f"{line.name}:{line.description}。inputSchema:{line.inputSchema}" for line in response.tools)
#             session_index=session_index|{line.name:idx for line in response.tools}
#             tool_descriptions+=descriptions
#     except Exception as e:
#         print(f"错误: {e}")
#     finally:
#         await asyncio.gather(*(close_mcp_session(**e) for e in sessions))
#     return tool_descriptions,session_index


# async def mcp_tool_run(name,input_dict,server_params_list,session_index):
#     try:
#         sessions = await asyncio.gather(*(mcp_client(p) for p in server_params_list))
#         session = sessions[session_index[name]]['session']
#         content = await session.call_tool(name,input_dict)
#     except Exception as e:
#         print(f"错误: {e}")
#     finally:
#         await asyncio.gather(*(close_mcp_session(**e) for e in sessions))
#     return content.content[0].text

In [56]:
session_data=await mcp_client(server_params_list[0])

In [77]:
sessions = await asyncio.gather(*(mcp_client(p) for p in server_params_list))

In [55]:
await sessions[0].get('read').aclose()
await sessions[0].get('write').aclose()

In [69]:
# print(await sessions[3].get('session').list_tools())

In [83]:
result=await sessions[3].get('session').call_tool("get_all_documents",{})

In [328]:
# json.loads(result.content[0].text)

In [61]:
async def close_mcp_session(session, read, write, client):
    try:
        await client.__aexit__(None, None, None)
        await session.__aexit__(None, None, None)
    except:
        await read.aclose()
        await write.aclose()

In [74]:
await asyncio.gather(*(close_mcp_session(**p) for p in sessions))

[None, None, None, None]

In [60]:
# await session_data['session'].list_tools()

In [64]:
await close_mcp_session(**session_data)

In [30]:
server_params_list[0]

StdioServerParameters(command='npx', args=['--yes', '@modelcontextprotocol/server-filesystem', '/oper/ch/mcp_work'], env=None, cwd=None, encoding='utf-8', encoding_error_handler='strict')

In [81]:
# session_data

In [45]:
# tool_descriptions,session_index=await get_tool_description(server_params_list)

In [77]:
# content=await mcp_tool_run('read_file',{'path':'/oper/ch/code/test.txt'},server_params_list,session_index)

In [1]:
from mcp_client_claude import MCPClientManager

In [98]:
server_params_list = [StdioServerParameters(**{"command":"uv","args":["run","/oper/ch/code/chromadb_mcp.py","--ollama_model","bge-m3","--collection_name","my_ollama_documents"]})]

In [5]:
# server_params_list = [
#     StdioServerParameters(command="npx", args=["--yes", "@modelcontextprotocol/server-filesystem", '/oper/ch/code']),
#     StdioServerParameters(command="uvx", args=["mcp-server-fetch"]),
#     StdioServerParameters(command="npx", args=["-y","@modelcontextprotocol/server-puppeteer"])
# ]
manager = MCPClientManager(server_params_list)

In [6]:
descriptions, _ = await manager.initialize()

In [198]:
tools=await session['session'].list_tools()

In [211]:
for tool in tools.tools:
    print(f"{tool.name}:{tool.description}\n{tool.inputSchema}")

In [189]:
def simplify_type(schema: dict) -> str:
    # 优先解析 anyOf
    if "anyOf" in schema:
        types = [t.get("type", "any") for t in schema["anyOf"] if t.get("type") != "null"]
        return simplify_type({"type": types[0]}) + " (optional)" if "null" in [t.get("type") for t in schema["anyOf"]] else simplify_type({"type": types[0]})
    
    t = schema.get("type", "any")

    # array with item types
    if t == "array":
        item_type = schema.get("items", {}).get("type", "any")
        return f"List[{map_type(item_type)}]"

    return map_type(t)

def map_type(json_type: str) -> str:
    return {
        "string": "str",
        "integer": "int",
        "number": "float",
        "boolean": "bool",
        "object": "dict",
        "array": "list",
        "null": "None"
    }.get(json_type, "any")

def convert_schema_to_input_spec(schema: dict) -> dict:
    props = schema.get("properties", {})
    return {key: simplify_type(value) for key, value in props.items()}

In [210]:
convert_schema_to_input_spec(tool.inputSchema)

{}

In [5]:
from ollama import Client
client = Client(host='http://localhost:11434')

In [6]:
system_prompt="""
请根据输入的工具使用描述语言，输出查询使用工具的查询语言。
输入:delete_current_collection:删除当前使用的集合
输出： 删除集合、清除集合、移除当前集合、删除当前数据集
"""

In [270]:
response = client.chat(
    model="gemma3:27b",
    messages=[
        {"role": "system", "content": system_prompt},{"role":"user","content":'Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.'}
    ]
)

In [271]:
response.message.content

'读取文件内容、加载文件、打开文件、查看文件、获取文件内容、文件内容读取、读取文本文件、检查文件内容、访问文件内容\n'

In [10]:
import json

In [56]:
async def mcptools2database(server_params_list):
    metadata_list=[]
    documents=[]
    # for server_params in server_params_list:
    try:
        manager = MCPClientManager(server_params_list)
        # session=await manager._create_mcp_client(server_params)
        # tools=await session['session'].list_tools()
        for p in server_params_list:
            async with manager._create_session(p) as session_data:
                tools = await session_data['session'].list_tools()
                for tool in tools.tools:
                    print(f"{tool.name}工具正在登记")
                    # print(f"{tool.name}:{tool.description}\n{tool.inputSchema}")
                    response = client.chat(model="gemma3:27b",
                                           messages=[{"role": "system", "content": system_prompt},{"role":"user","content":f"输入：{tool.name}:{tool.description}\n输出："}])
                    documents.append(tool.description)
                    metadata_list.append({"tool_name": tool.name,
                                          "description": tool.description,
                                          "input_spec":json.dumps(convert_schema_to_input_spec(tool.inputSchema)),
                                           "tags": f"{response.message.content}"})
    except Exception as e:
        print(str(e))
    finally:
        await manager.close_all()
    return documents,metadata_list

In [ ]:
async def mcp_run_tool(name,input_dict,server_params_list):
    try:
        manager = MCPClientManager(server_params_list)
        descriptions, _ = await manager.initialize()
        result = await manager.run_tool(name,input_dict)
    finally:
        await manager.close_all()
    return result

In [49]:
async def mcptools2database(server_params_list):
    metadata_list = []
    documents = []
    manager = None
    
    try:
        manager = MCPClientManager(server_params_list)
        print("MCPClientManager created, initializing connections...")
        
        # Process each server parameter individually instead of using the manager's initialize method
        for i, p in enumerate(server_params_list):
            try:
                print(f"Connecting to server {i+1}/{len(server_params_list)}: {p.command} {' '.join(p.args)}")
                async with manager._create_session(p) as session_data:
                    print(f"Connected to server {i+1}, listing tools...")
                    tools = await session_data['session'].list_tools()
                    print(f"Found {len(tools.tools)} tools in server {i+1}")
                    
                    for tool in tools.tools:
                        print(f"Registering tool: {tool.name}")
                        documents.append(tool.description)
                        metadata_list.append({
                            "tool_name": tool.name,
                            "description": tool.description,
                            "input_schema": tool.inputSchema,
                        })
            except Exception as e:
                print(f"Error processing server {i+1}: {p.command} {' '.join(p.args)}")
                print(f"Error details: {str(e)}")
                # Continue with next server instead of failing completely
        
    except Exception as e:
        print(f"Failed to create MCPClientManager: {str(e)}")
    finally:
        if manager:
            print("Closing all connections...")
            await manager.close_all()
            
    return documents, metadata_list

In [57]:
# documents,metadata_list=await mcptools2database(server_params_list)

In [59]:
len(documents)

32

In [336]:
import time 

In [338]:
# doc_list=[]
# metadatas_list=[]
# for line in server_params_list:
#     documents,metadata_list=await mcptools2database([line])
#     doc_list.extend(documents)
#     metadatas_list.extend(metadata_list)
#     time.sleep(5)
#     # await mcp_run_tool("add_documents",{"documents":documents,"metadatas":metadata_list},server_params)

In [31]:
# documents,metadata_list=await mcptools2database(server_params_list)

In [12]:
server_params = [StdioServerParameters(command="uv",args=["run","/oper/ch/code/chromadb_mcp.py","--ollama_model","bge-large-zh-v1.5:f16","--collection_name","tools_collection"])]

In [ ]:
curl http://localhost:11434/api/embed -d '{
  "model": "bge-large-zh-v1.5:f16",
  "input": "Llamas are members of the camelid family"
}'

In [69]:
await mcp_run_tool("add_documents",{"documents":documents,"metadatas":metadata_list},server_params)

All sessions closed


'44e43440-f3f3-47da-a6f1-07f53d1d661d'

In [302]:
result=await mcp_run_tool("get_all_documents",{},server_params)

In [67]:
await mcp_run_tool("delete_current_coll"get_all_documents",{}ection",None,server_params)

All sessions closed


'成功删除集合: tools_collection'

In [70]:
result=await mcp_run_tool("search_documents",{"query":"写入文件`/oper/ch/work/test.txt`","include":["documents", "metadatas"] },server_params)

All sessions closed


In [73]:
# json.loads(result)

In [319]:
# curl http://localhost:11434/api/embed -d '{
#   "model": "snowflake-arctic-embed2:latest",
#   "input": "Llamas are members of the camelid family"
# }'

In [304]:
# json.loads(result)

In [161]:
# 将字典保存为 JSON 文件
def save_dict_to_json(data, file_path):
    """
    将字典保存为 JSON 文件。
    
    :param data: 要保存的字典
    :param file_path: JSON 文件的路径
    """
    try:
        # 打开文件并写入 JSON 数据
        with open(file_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f"字典已成功保存到 {file_path}")
    except Exception as e:
        print(f"保存字典时出错: {e}")

In [180]:
save_dict_to_json(input_data, "/oper/ch/mcp_work/filesystem_tools.json")

字典已成功保存到 /oper/ch/mcp_work/filesystem_tools.json


In [163]:
# input_data

In [278]:
# document = [
#     "添加文档到ChromaDB集合"
# ]
# metadata=[
#     {
#         "service_name": "chromadb_service",
#         "service_params": {
#             "command": "uv",
#             "args": ["run", "/oper/ch/code/chromadb_mcp.py"]
#         },
#         "tool_name": "add_documents",
#         "tool_usage": "添加文档到 ChromaDB 集合\n\nArgs:\n    documents: 文本内容列表\n    metadatas: 元数据列表（可选）\n    ids: ID列表（可选，如不提供将自动生成）\n\nReturns:\n    添加的文档ID列表",
#         "inputSchema": {
#             "type": "object",
#             "title": "add_documentsArguments",
#             "required": ["documents"],
#             "properties": {
#                 "documents": {"type": "array", "items": {"type": "string"}, "title": "Documents"},
#                 "metadatas": {
#                     "anyOf": [{"type": "array", "items": {"type": "object"}}, {"type": "null"}],
#                     "default": None, "title": "Metadatas"
#                 },
#                 "ids": {
#                     "anyOf": [{"type": "array", "items": {"type": "string"}}, {"type": "null"}],
#                     "default": None, "title": "Ids"
#                 }
#             }
#         }
#     }]

In [172]:
def prepare_metadata(tool_item: dict):
    return {
        "tool_name": tool_item["tool_name"],
        "tool_usage": tool_item["tool_usage"],
        "inputSchema": json.dumps(tool_item["inputSchema"]),
        "service_name": tool_item["service_name"],
        "service_params": json.dumps(tool_item["service_params"]),
    }

In [175]:
input_data={"documents":documents,"metadatas":[prepare_metadata(line)  for line in metadata]}

In [260]:
# print(descriptions)

In [177]:
result = await manager.run_tool("add_documents",input_data)

In [212]:
result = await manager.run_tool("get_collection_info",None)

In [214]:
result = await manager.run_tool("delete_current_collection",None)

In [215]:
print(result)

{"name": "tools_collection", "count": 24}


In [118]:
server_params

[StdioServerParameters(command='uv', args=['run', '/oper/ch/code/chromadb_mcp.py', '--ollama_model', 'bge-large-zh-v1.5:f16', '--collection_name', 'tools_collection'], env=None, cwd=None, encoding='utf-8', encoding_error_handler='strict')]

In [19]:
from pydantic import BaseModel
from enum import Enum
from typing import Dict, Any, List, Union, Optional, Literal
import json,re

In [101]:
# 定义数据模型
from pydantic import BaseModel
from enum import Enum
from typing import Dict, Any, List, Union, Optional, Literal
    
class ActionInfo(BaseModel):
    action_name: str
    action_type: Union[Tool, Instruction]

class WorkFlow(BaseModel):
    task_name: str
    workflow: List[ActionInfo]

# ========== 期望输出 ==========
class DesireOutput(BaseModel):
    expected_result: Optional[str] = None          # For Instruction
    expected_data: Optional[Dict[str, Any]] = None # For Tool
    expected_workflow_result: Optional[str] = None # For WorkFlow

# ========== 评估模型 ==========
class InstructionEval(BaseModel):
    correct: bool
    feedback: Optional[str] = None

class ToolEval(BaseModel):
    success: bool
    result_summary: Optional[str] = None
    error: Optional[str] = None

class WorkFlowEval(BaseModel):
    completed_steps: int
    total_steps: int
    summary: Optional[str] = None

class Evaluation(BaseModel):
    evaluation_type: Literal["instruction", "tool", "workflow"]
    instruction_eval: Optional[InstructionEval] = None
    tool_eval: Optional[ToolEval] = None
    workflow_eval: Optional[WorkFlowEval] = None
    next_goal: Optional[str] = None
    last_step: Optional[bool] = False

# ========== 步骤定义 ==========
class StepInfo(BaseModel):
    step_id: int
    step_name: str
    step_description: str
    actions_name: List[str]
    actions: List[Union[Instruction, Tool, WorkFlow]]
    desire_output: DesireOutput
    evaluation: Evaluation


In [7]:
class Tool(BaseModel):
    name: str
    arguments: Dict[str, Any]
    action: str
    async def execute(self) -> Dict[str, Any]:
        try:
            descriptions,_= await manager.initialize()
            response = await manager.run_tool(self.name,self.arguments)
            await manager.close_all()
            if response:
                return {"tool_name": self.name,
                        "info": f"Executed {self.action} with {self.arguments}",
                        "result": response
                       }
            else:
                  return {"tool_name": self.name,
                          "info": f"Executed {self.action} with {self.arguments}",
                          "result": "tool call is success. "
                       }
        except Exception as e:
            return  {"tool_name": self.name,
                    "info": f"Executed {self.action} with {self.arguments}",
                    "error": str(e)
                   }

In [377]:
tool=Tool(**{"name": "fetch", "arguments": {"url": "https://github.com/browser-use/browser-use/blob/main/browser_use/agent/system_prompt.md","raw":False}, "action": "查询天气预报"})

In [378]:
response = await tool.execute()

In [380]:
# print(response['result'])

In [1]:
from ollama import AsyncClient

In [36]:
# client= AsyncClient("http://localhost:11434")

In [37]:
# response = await client.chat(model="gemma3:27b",messages=[{"role":'user',"content":'hi'}])

In [279]:
async def Ollama_Asyncclient(query,host,model,prompt=None,format_output="json"):
    client = AsyncClient(host)
    if not prompt:
        message = [{'role': 'user', 'content': query}]
    else:
         message = [{'role':'system','content':prompt},{'role': 'user', 'content': query}]
    response = await client.chat(model = model ,messages = message, format= format_output)
    return response 

In [274]:
class Instruction(BaseModel):
    prompt: Optional[str] = None  
    query: str
    async def execute(self,model="qwen3:4b-fp16",host="http://localhost:11434",format_output='json'):
        try:
        
            response = await Ollama_Asyncclient(query = self.query,
                                                prompt = self.prompt,
                                                host = host,
                                                model = model,
                                                format_output = format_output
                                                )
         
            return {"result": response}
        except Exception as e: 
            return {"error": f"{e}" }

In [135]:
tool=Tool(name="search_documents",arguments={"query":"请作一首诗描写深圳湾海景,然后把诗写入文件`/oper/ch/work/test.txt`","include":["documents", "metadatas"],"n_results":3},action="写入文件")

In [136]:
result=await tool.execute()

In [142]:
tool_descriptions="\n".join([ f"{line['tool_name']}:{line['description']}input_spec:{line['input_spec']}"for line in json.loads(result["result"])['metadatas'][0]])

In [138]:
format_output=get_format_instructions(Tool)

In [144]:
await manager.close_all()

In [141]:
print(descriptions)

edit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}
get_document_count:获取集合中文档的数量
    
    Returns:
        文档数量
    input_spec:{}
get_collection_info:获取集合信息
    
    Returns:
        集合信息
    input_spec:{}


In [160]:
query=f"""
你可以调用以下工具来完成任务：
{tool_descriptions}

请根据用户的请求选择最合适的工具来完成。如果**没有任何工具**可以合理完成请求，请直接返回字符串 `没有合适的工具完成给定任务`，不要尝试编造或虚构实现方式。

用户的请求是：
"请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/work/test.txt`"

请按以下格式返回结果：
{format_output}
"""

In [329]:
# print(tool_descriptions)

In [155]:
instruction=Instruction(query=query)

In [156]:
response=await instruction.execute()

In [157]:
response['result']['message']['content']

'{"name": "write_file", "arguments": {"path": "/oper/ch/work/test.txt", "content": "深圳湾畔景，碧波映霞光。\\n海鸥翩跹舞，浪花逐夕阳。\\n高楼林立起，科技耀鹏程。\\n南国春常在，繁华似梦境。"}, "action": "write_file"}'

In [162]:
test=json.loads(response['result']['message']['content'])

In [235]:
ToolPrompt="""
你将收到一个用户请求为: `user_request` 和一个上下文 `knowledge_context`，请根据用户请求和上下文的格式和内容，判断属于以下哪种情况，并严格按照指定格式返回 JSON 结果：
---
### 🔷 **第一种情况：用户请求匹配的是知识库工具的使用描述**

例如如下形式：

```
edit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}
get_document_count:获取集合中文档的数量    
    Returns:
        文档数量
    input_spec:{}
get_collection_info:获取集合信息   
    Returns:
        集合信息
    input_spec:{}
```

- 请根据工具能力与用户请求匹配程度，选择**最合适的工具**完成任务。
- 并生成如下格式的 JSON：

```json
{
  "name": "工具名",
  "arguments": { 工具所需参数 },
  "action": "简洁描述工具执行的动作"
}
```
### 🔷 **第二种情况：用户请求匹配的是知识库完整的任务示例**

例如如下形式：

```json
{
  "action_name": "请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`",
  "action_type": {
    "name": "write_file",
    "arguments": {
      "path": "/oper/ch/mcp_work/test.txt",
      "content": "深圳湾畔映晴空..."
    },
    "action": "将用户指定的诗歌写入到文件中"
  }
}
```

- 请仿照其结构与风格，根据用户的新请求生成对应 JSON，字段说明如下：
  - `action_name`：简洁复述任务要求；
  - `action_type.name`：工具名（如 write_file）；
  - `action_type.arguments`：包含路径和要写入的文本；
  - `action_type.action`：一句话描述任务动作。

---

### 🔷 **第三种情况：知识库与问题无关或无法提供支持**

- 如果上下文内容与工具调用无关（例如是 API 文档、知识性描述等），或者没有任何匹配的执行方式，请直接返回：

```
没有工具可以提供使用
```
---

### 🔸 **其他要求：**

- 所有 JSON 字符串必须严格合法（双引号、转义符等符合规范）；
- 不输出除 JSON 结果外的任何额外说明；
- 所有换行请使用 `\n`；
- 如果生成内容较长，如诗句，请保持内容符合主题美感；
"""


In [335]:
# print(ToolPrompt+ f"""用户请求为:{query}\n知识库上下文为:{description}""")
# print(description)

In [ ]:
# prompt=f"""
#     你只能选择以下工具来完成任务,不能捏造不存在的工具来完成任务：
    
#     {tool_descriptions}
    
#     用户的请求是：{query}
    
#     {format_prompt}
#     """
# print(prompt)

In [349]:
# async def llm2tool(query,n_results=10)
#     excution_log = '模型调用工具流程\nStep1:\n查询任务可用工具\n'
    
#     input_dict={"name":"search_documents",
#                "arguments":{"query":query,
#                          "include":["documents", "metadatas"],
#                          "n_results":n_results},
#                 "action":f"查询工具,执行任务:{query}"
#                }
    
#     excution_log += json.dumps(input_dict,ensure_ascii=False)
    
#     tool=Tool(**input_dict)
    
#     result=await tool.execute()
    
#     if "result" in result:
#         result=json.loads(result["result"])['metadatas'][0]
#         tool_descriptions="\n".join([ f"{line['tool_name']}:{line['description']}input_spec:{line['input_spec']}"for line in result])
#         # print(tool_descriptions)
#         excution_log += f"\n查询知识库发现{n_results}个工具与执行任务相关."
        
#         format_prompt=get_format_instructions(Tool)
#     else:
#         return {"excution_log":excution_log+f"\n查询工具,报错为:{result['error']}"}
    
#     prompt=ToolPrompt+f"用户请求为:{query}\n知识库上下文为:{tool_descriptions}"
    
#     excution_log += f"\nStep2:\n大模型选择一个工具辅助执行任务.\n大模型prompt为:\n{prompt}"
    
#     instruction=Instruction(query=prompt)
    
#     response=await instruction.execute()
#     try:
#         tool_json=json.loads(response['result']['message']['content'])
#     except :
#         # print("大模型返回数据格式不是标准的json格式,使用正则表达式提取json")
#         excution_log += "\n大模型返回数据格式不是标准的json格式,使用正则表达式提取json"
        
#         tool_json=extract_tool_format_from_string(response['result']['message']['content'])
        
#     if tool_json:
#         excution_log += f"\n大模型挑选执行任务的工具:{json.dumps(tool_json,ensure_ascii=False)}\nStep3:\n使用工具执行任务."
#         # print(tool_json)
#         try:
#             tool_call=Tool(**tool_json)
#         except:
#             tool_call=Tool(**tool_json['action_type'])
#         call_result = await tool_call.execute()
        
#         if "result" in call_result :
#             excution_log += f"\n工具执行结果为:\n{call_result['result']}"
#         else:
#             excution_log += f"\n工具执行失败,报错为:\n{call_result['error']}"
            
#         action = ActionInfo(action_name=query,action_type=tool_call)
#     else:
#         excution_log += "\n正则表达式无法从大模型返回的字符串:{tool_json},提取有效的json格式,工具运行失败."
        
#         action = f"tool_call faild,no tool can be use "
    
#     return {"excution_log":excution_log,"action": action}    

In [ ]:
class LLM2Tool:
    """
    A class that encapsulates the process of using LLM to select and execute appropriate tools
    based on a user query.
    """
    
    def __init__(self, tool_class, instruction_class, actioninfo_class, tool_prompt):
        """
        Initialize the LLM2Tool class.
        
        Args:
            tool_class: The Tool class to use for executing tools
            instruction_class: The Instruction class for interacting with the LLM
            actioninfo_call: The ActionInfo Class record tools use 
            tool_prompt: The prompt template to use when asking LLM to select a tool
            # format_instructions_func: Function to get format instructions for Tool class
        """
        self.Tool = tool_class
        self.Instruction = instruction_class
        self.ActionInfo = actioninfo_class
        self.ToolPrompt = tool_prompt
        # self.get_format_instructions = format_instructions_func
        self.execution_log = ""
        
    async def step1_search_available_tools(self, query: str, n_results: int = 10) -> Dict[str, Any]:
        """
        Step 1: Search for available tools that are relevant to the user query.
        
        Args:
            query: The user query
            n_results: Number of tool results to retrieve\n根据提供的工具使用示例和新问题：\n保持action_type.name与示例一致；\n根据新问题，改写：\naction_name（简明描述新任务）；\naction_type.arguments（参数内容应与新问题一致，如文件路径、内容等）；\naction_type.action（用简洁语言描述实际动作）；\n工具使用示例:action_name=\'请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`\' action_type=Tool(name=\'write_file\', arguments={\'path\': \'/oper/ch/mcp_work/test.txt\', \'content\': \'深圳湾畔碧波连，\\n万象澄明映晴天。\\n白鹭翩跹逐浪远，\\n轻舟荡漾戏鸥闲。\\n长桥卧浪通霄汉，\\n灯火璀璨耀海山。\\n此景只应仙界有，\\n人间难得几回观。\'}, action="Create or overwrite a file with the poem about Shenzhen Bay\'s seascape")\n新问题:请作一首诗描写春天山景,然后把诗写入文件 `/oper/ch/mcp_work/tmp.txt`\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Instruction": {"properties": {"prompt": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Prompt"}, "query": {"title": "Query", "type": "string"}}, "required": ["query"], "title": "Instruction", "type": "object"}, "Tool": {"properties": {"name": {"title": "Name", "type": "string"}, "arguments": {"additionalProperties": true, "title": "Arguments", "type": "object"}, "action": {"title": "Action", "type": "string"}}, "required": ["name", "arguments", "action"], "title": "Tool", "type": "object"}}, "properties": {"action_name": {"title": "Action Name", "type": "string"}, "action_type": {"anyOf": [{"$ref": "#/$defs/Tool"}, {"$ref": "#/$defs/Instruction"}], "title": "Action Type"}}, "required": ["action_name", "action_type"]}\n```\n只输出符合规范的 JSON，不输出其他文字或解释。\n    output: Any
            
        Returns:
            Dictionary containing tool descriptions or error information
        """
        self.execution_log = '模型调用工具流程\nStep1:\n查询任务可用工具'
        
        input_dict = {
            "name": "search_documents",
            "arguments": {
                "query": query,
                "include": ["documents", "metadatas"],
                "n_results": n_results
            },
            "action": f"查询工具,执行任务:{query}"
        }
        
        self.execution_log = self.execution_log + '\n' + json.dumps(input_dict, ensure_ascii=False)
        
        tool = self.Tool(**input_dict)
        result = await tool.execute()
        
        if "result" in result:
            result_data = json.loads(result["result"])['metadatas'][0]
            tool_descriptions = "\n".join([
                f"{line['tool_name']}:{line['description']}input_spec:{line['input_spec']}" 
                for line in result_data
            ])
            self.execution_log += f"\n查询知识库发现{n_results}个工具与执行任务相关."
            # format_prompt = self.get_format_instructions(self.Tool)
            
            return {
                "success": True,
                "tool_descriptions": tool_descriptions,
                # "format_prompt": format_prompt
            }
        else:
            self.execution_log += f"\n查询工具,报错为:{result['error']}"
            return {
                "success": False,
                "error": result['error']
            }
    
    async def step2_llm_select_tool(self, query: str, tool_descriptions: str) -> Dict[str, Any]:
        """
        Step 2: Use LLM to select the most appropriate tool based on user query and available tools.
        
        Args:
            query: The user query
            tool_descriptions: String describing available tools and their specifications
            
        Returns:
            Dictionary containing selected tool information or error
        """
        prompt = self.ToolPrompt + f"用户请求为:{query}\n知识库上下文为:{tool_descriptions}"
        
        self.execution_log += f"\nStep2:\n大模型选择一个工具辅助执行任务.\n大模型prompt为:\n{prompt}"
        
        instruction = self.Instruction(query=prompt)
        response = await instruction.execute()
        
        try:
            tool_json = json.loads(response['result']['message']['content'])
        except:
            self.execution_log += "\n大模型返回数据格式不是标准的json格式,使用正则表达式提取json"
            tool_json = self.extract_tool_format_from_string(response['result']['message']['content'])
        
        if tool_json:
            self.execution_log += f"\n大模型挑选执行任务的工具:{json.dumps(tool_json, ensure_ascii=False)}"
            return {
                "success": True,
                "tool_json": tool_json
            }
        else:
            self.execution_log += "\n正则表达式无法从大模型返回的字符串提取有效的json格式,工具运行失败."
            return {
                "success": False,
                "error": "Failed to extract valid JSON from LLM response"
            }
    
    async def step3_execute_tool(self, tool_json: Dict[str, Any]) -> Dict[str, Any]:
        """
        Step 3: Execute the selected tool with the provided parameters.
        
        Args:
            tool_json: The tool specification and parameters selected by the LLM
            
        Returns:
            Dictionary containing execution results or error information
        """
        self.execution_log += "\nStep3:\n使用工具执行任务."
        
        try:
            try:
                tool_call = self.Tool(**tool_json)
            except:
                tool_call = self.Tool(**tool_json['action_type'])
                
            call_result = await tool_call.execute()
            
            if "result" in call_result:
                self.execution_log += f"\n工具执行结果为:\n{call_result['result']}"
                return {
                    "success": True,
                    "result": call_result['result'],
                    "tool_call": tool_call
                }
            else:
                self.execution_log += f"\n工具执行失败,报错为:\n{call_result['error']}"
                return {
                    "success": False,
                    "error": call_result['error'],
                    "tool_call": tool_call
                }
        except Exception as e:
            self.execution_log += f"\n工具执行过程中发生异常:{str(e)}"
            return {
                "success": False,
                "error": str(e)
            }
    
    @staticmethod
    def extract_tool_format_from_string(text: str) -> Optional[Dict[str, Any]]:
        """
        从字符串中提取JSON数据并解析为符合ToolFormat结构的字典
        
        Args:
            text: 可能包含JSON数据的字符串
            
        Returns:
            如果成功解析，返回符合ToolFormat结构的字典，否则返回None
        """
        # 先尝试一种更简单的方式：找到所有可能的JSON起始位置
        json_start_positions = [m.start() for m in re.finditer(r'{"name":', text)]
        
        for start_pos in json_start_positions:
            # 从每个起始位置开始尝试解析
            partial_text = text[start_pos:]
            
            # 用递增的方式尝试解析不同长度的子字符串
            for end_pos in range(len(partial_text)):
                try:
                    # 尝试各种长度的子字符串，直到找到有效的JSON
                    possible_json = partial_text[:end_pos+1]
                    if possible_json.count('{') != possible_json.count('}'):
                        continue  # 花括号不平衡，继续尝试更长的子字符串
                        
                    data = json.loads(possible_json)
                    
                    # 验证是否符合ToolFormat结构
                    if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                        return data
                        
                except json.JSONDecodeError:
                    continue
                except Exception:
                    continue
                    
        # 如果上面的方法失败，尝试一个更通用的方法
        try:
            # 尝试找到最外层的大括号匹配
            brace_pattern = r'\{(?:[^{}]|(?:\{[^{}]*\}))*\}'
            matches = re.finditer(brace_pattern, text)
            
            for match in matches:
                try:
                    json_str = match.group(0)
                    data = json.loads(json_str)
                    
                    if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                        return data
                except:
                    continue
        except:
            pass
        
        # 最后一种方法：尝试使用更复杂的递归方式查找JSON
        def find_balanced_braces(s, start_index):
            """递归查找平衡的花括号"""
            stack = []
            for i in range(start_index, len(s)):
                if s[i] == '{':
                    stack.append(i)
                elif s[i] == '}':
                    if stack:
                        start = stack.pop()
                        if not stack:  # 最外层花括号已匹配完成
                            return s[start:i+1]
            return None
        
        # 查找所有可能的JSON开始位置
        for i in range(len(text)):
            if text[i] == '{':
                potential_json = find_balanced_braces(text, i)
                if potential_json:
                    try:
                        data = json.loads(potential_json)
                        if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                            return data
                    except:
                        continue
                        
        return None
        
    
    async def execute(self, query: str, n_results: int = 10) -> Dict[str, Any]:
        """
        Main method to execute the entire LLM2Tool workflow.
        
        Args:
            query: The user query
            n_results: Number of tool results to retrieve
            
        Returns:
            Dictionary containing execution log and action information
        """
        # Step 1: Search for available tools
        step1_result = await self.step1_search_available_tools(query, n_results)
        if not step1_result["success"]:
            return {"execution_log": self.execution_log, "action": f"""知识库查找工具失败.错误:{step1_result["error"]}""" }
        
        # Step 2: Have LLM select appropriate tool
        step2_result = await self.step2_llm_select_tool(query, step1_result["tool_descriptions"])
        if not step2_result["success"]:
            return {"execution_log": self.execution_log, "action": f"""大模型挑选工具执行失败.错误:{step2_result["error"]}"""}
        
        # Step 3: Execute the selected tool
        step3_result = await self.step3_execute_tool(step2_result["tool_json"])
        
        action_info = None
        if step3_result["success"]:
            
            action_info = self.ActionInfo(action_name=query, action_type=step3_result["tool_call"])
        else:
            action_info = "tool_call failed, no tool can be used"
        
        return {"execution_log": self.execution_log, "action": action_info}

In [344]:
llm2tool = LLM2Tool(Tool, Instruction,ActionInfo,ToolPrompt)

In [375]:
result = await llm2tool.execute("访问这个网站`https://weather.cma.cn/web/weather/59493.html`查询天气预报")

In [376]:
print(result['execution_log'])

模型调用工具流程
Step1:
查询任务可用工具
{"name": "search_documents", "arguments": {"query": "访问这个网站`https://weather.cma.cn/web/weather/59493.html`查询天气预报", "include": ["documents", "metadatas"], "n_results": 10}, "action": "查询工具,执行任务:访问这个网站`https://weather.cma.cn/web/weather/59493.html`查询天气预报"}
查询知识库发现10个工具与执行任务相关.
Step2:
大模型选择一个工具辅助执行任务.
大模型prompt为:

你将收到一个用户请求为: `user_request` 和一个上下文 `knowledge_context`，请根据用户请求和上下文的格式和内容，判断属于以下哪种情况，并严格按照指定格式返回 JSON 结果：
---
### 🔷 **第一种情况：用户请求匹配的是知识库工具的使用描述**

例如如下形式：

```
edit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}
get_document_count:获取集合中文档的数量    
    Returns:
        文档数量
    input_spec:{}
get_collection_info:获取集合信息   
    Returns:
        集合信息
    input_spec:{}
```

- 请根据工具能力与用户请求匹配程度，选择**最合适的工具**完成任务。
- 并生成如下格式的 JSON：

```json
{
  "name": "工具名",
  

In [319]:
result = await llm2tool("请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`")

In [324]:
result = await llm2tool("读取文件`/oper/ch/mcp_work/test.txt`")

In [336]:
# print(result['excution_log'])

In [310]:
print(result['action'])

action_name='获取知识库信息' action_type=Tool(name='get_collection_info', arguments={}, action='获取集合信息')


In [225]:
result['action'].model_dump()['action_type']

{'name': 'write_file',
 'arguments': {'path': '/oper/ch/mcp_work/test.txt',
  'content': '深圳湾海景诗\n潮起潮落映日红，\n海天一色共长空。\n渔舟唱晚霞光里，\n鹭影翩翩舞碧穹。\n风送花香盈袖暖，\n浪拍礁石碎玉融。\n此景只应天上见，\n人间何处不玲珑。'},
 'action': 'write_file'}

In [238]:
# convert_schema_to_input_spec(ActionInfo.model_json_schema())

In [193]:
# ActionInfo.model_json_schema()

In [146]:
result['action']

ActionInfo(action_name='请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`', action_type=Tool(name='write_file', arguments={'path': '/oper/ch/mcp_work/test.txt', 'content': '深圳湾畔映晴空，碧波荡漾接远峰。\n白帆点点逐浪去，鸥鸟翩翩戏水踪。\n两岸灯火映夜色，一桥飞架连西东。\n此间胜景谁人绘？天然画卷在心中。'}, action='将用户指定的诗歌写入到文件中'))

In [132]:
# result['excution_log']

In [264]:
# instruction.model_dump()

In [297]:
# eval_prompt

In [382]:
async def ResultEval(execution_log):
    eval_prompt=f"""
        请根据工具执行的日志返回评估结果.
        输入的工具执行日志为:
        {execution_log}
        摘要的模板为:
        ```【工具执行评估】
        1. 任务目标：_______
        2. 选用工具：
           - 工具名称：_______
           - 是否符合任务？（是/否）
        3. 工具调用参数：
           - 参数正确？（是/否）
        4. 工具执行结果：
           - 返回信息：_______
           - 是否成功？（是/否）
        5. 最终结论：工具执行（成功/失败），原因：_______```
        {get_format_instructions(ToolEval)}
        """
    instruction = Instruction(query=eval_prompt)
    result_eval = await instruction.execute()
    text = result_eval['result']['message']['content']
    cleaned = re.sub(r"<think>[\s\S]*?<\/think>", "", text)
    return cleaned

In [383]:
eval_result = await ResultEval(result['execution_log'])

In [390]:
eval_prompt=f"""
        请根据工具执行的日志返回评估结果.
        输入的工具执行日志为:
        {result['execution_log']}
        摘要的模板为:
        ```【工具执行评估】
        1. 任务目标：_______
        2. 选用工具：
           - 工具名称：_______
           - 是否符合任务？（是/否）
        3. 工具调用参数：
           - 参数正确？（是/否）
        4. 工具执行结果：
           - 返回信息：_______
           - 是否成功？（是/否）
        5. 最终结论：工具执行（成功/失败），原因：_______```
        {get_format_instructions(ToolEval)}
        """

In [391]:
eval_prompt

'\n        请根据工具执行的日志返回评估结果.\n        输入的工具执行日志为:\n        模型调用工具流程\nStep1:\n查询任务可用工具\n{"name": "search_documents", "arguments": {"query": "访问这个网站`https://weather.cma.cn/web/weather/59493.html`查询天气预报", "include": ["documents", "metadatas"], "n_results": 10}, "action": "查询工具,执行任务:访问这个网站`https://weather.cma.cn/web/weather/59493.html`查询天气预报"}\n查询知识库发现10个工具与执行任务相关.\nStep2:\n大模型选择一个工具辅助执行任务.\n大模型prompt为:\n\n你将收到一个用户请求为: `user_request` 和一个上下文 `knowledge_context`，请根据用户请求和上下文的格式和内容，判断属于以下哪种情况，并严格按照指定格式返回 JSON 结果：\n---\n### 🔷 **第一种情况：用户请求匹配的是知识库工具的使用描述**\n\n例如如下形式：\n\n```\nedit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}\nget_document_count:获取集合中文档的数量    \n    Returns:\n        文档数量\n    input_spec:{}\nget_collection_info:获取集合信息   \n    Returns:\n        集合信息\n    input_spec:{}\n```\n

In [388]:
# json.loads(eval_result)['result_summary']

In [128]:
def ReuseToolPrompt(action,query):
    actioninfo=get_format_instructions(ActionInfo)
    return f"""
根据提供的工具使用示例和新问题：
保持action_type.name与示例一致；
根据新问题，改写：
action_name（简明描述新任务）；
action_type.arguments（参数内容应与新问题一致，如文件路径、内容等）；
action_type.action（用简洁语言描述实际动作）；
工具使用示例:{action}
新问题:{query}
{actioninfo}
只输出符合规范的 JSON，不输出其他文字或解释。
"""

In [153]:
result['action'].model_dump_json()

'{"action_name":"请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`","action_type":{"name":"write_file","arguments":{"path":"/oper/ch/mcp_work/test.txt","content":"深圳湾畔映晴空，碧波荡漾接远峰。\\n白帆点点逐浪去，鸥鸟翩翩戏水踪。\\n两岸灯火映夜色，一桥飞架连西东。\\n此间胜景谁人绘？天然画卷在心中。"},"action":"将用户指定的诗歌写入到文件中"}}'

In [157]:
print(tool_descriptions)

edit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}
get_document_count:获取集合中文档的数量
    
    Returns:
        文档数量
    input_spec:{}
get_collection_info:获取集合信息
    
    Returns:
        集合信息
    input_spec:{}


In [159]:
print(ReuseToolPrompt(result['action'].model_dump_json(),'请作一首诗描写春天山景,然后把诗写入文件 `/oper/ch/mcp_work/tmp.txt`'))


根据提供的工具使用示例和新问题：
保持action_type.name与示例一致；
根据新问题，改写：
action_name（简明描述新任务）；
action_type.arguments（参数内容应与新问题一致，如文件路径、内容等）；
action_type.action（用简洁语言描述实际动作）；
工具使用示例:{"action_name":"请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`","action_type":{"name":"write_file","arguments":{"path":"/oper/ch/mcp_work/test.txt","content":"深圳湾畔映晴空，碧波荡漾接远峰。\n白帆点点逐浪去，鸥鸟翩翩戏水踪。\n两岸灯火映夜色，一桥飞架连西东。\n此间胜景谁人绘？天然画卷在心中。"},"action":"将用户指定的诗歌写入到文件中"}}
新问题:请作一首诗描写春天山景,然后把诗写入文件 `/oper/ch/mcp_work/tmp.txt`
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Instruction": {"properties": {"prompt": {"anyOf": [{"type":

In [165]:
query='请作一首诗描写春天山景,然后把诗写入文件 `/oper/ch/mcp_work/tmp.txt`'

In [169]:
description=tool_descriptions+'\n'+result['action'].model_dump_json()

In [170]:
description

'edit_file:Make line-based edits to a text file. Each edit replaces exact line sequences with new content. Returns a git-style diff showing the changes made. Only works within allowed directories.input_spec:{"path": "str", "edits": "List[dict]", "dryRun": "bool"}\nget_document_count:获取集合中文档的数量\n    \n    Returns:\n        文档数量\n    input_spec:{}\nget_collection_info:获取集合信息\n    \n    Returns:\n        集合信息\n    input_spec:{}\n{"action_name":"请作一首诗描写深圳湾海景,然后把诗写入文件 `/oper/ch/mcp_work/test.txt`","action_type":{"name":"write_file","arguments":{"path":"/oper/ch/mcp_work/test.txt","content":"深圳湾畔映晴空，碧波荡漾接远峰。\\n白帆点点逐浪去，鸥鸟翩翩戏水踪。\\n两岸灯火映夜色，一桥飞架连西东。\\n此间胜景谁人绘？天然画卷在心中。"},"action":"将用户指定的诗歌写入到文件中"}}'

In [126]:
# print(result['excution_log'])print(result['excution_log'])

In [338]:
import json
import re
from typing import Dict, Any, Optional

def extract_tool_format_from_string(text: str) -> Optional[Dict[str, Any]]:
    """
    从字符串中提取JSON数据并解析为符合ToolFormat结构的字典
    
    Args:
        text: 可能包含JSON数据的字符串
        
    Returns:
        如果成功解析，返回符合ToolFormat结构的字典，否则返回None
    """
    # 先尝试一种更简单的方式：找到所有可能的JSON起始位置
    json_start_positions = [m.start() for m in re.finditer(r'{"name":', text)]
    
    for start_pos in json_start_positions:
        # 从每个起始位置开始尝试解析
        partial_text = text[start_pos:]
        
        # 用递增的方式尝试解析不同长度的子字符串
        for end_pos in range(len(partial_text)):
            try:
                # 尝试各种长度的子字符串，直到找到有效的JSON
                possible_json = partial_text[:end_pos+1]
                if possible_json.count('{') != possible_json.count('}'):
                    continue  # 花括号不平衡，继续尝试更长的子字符串
                    
                data = json.loads(possible_json)
                
                # 验证是否符合ToolFormat结构
                if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                    return data
                    
            except json.JSONDecodeError:
                continue
            except Exception:
                continue
                
    # 如果上面的方法失败，尝试一个更通用的方法
    try:
        # 尝试找到最外层的大括号匹配
        brace_pattern = r'\{(?:[^{}]|(?:\{[^{}]*\}))*\}'
        matches = re.finditer(brace_pattern, text)
        
        for match in matches:
            try:
                json_str = match.group(0)
                data = json.loads(json_str)
                
                if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                    return data
            except:
                continue
    except:
        pass
        
    # 最后一种方法：尝试使用更复杂的递归方式查找JSON
    def find_balanced_braces(s, start_index):
        """递归查找平衡的花括号"""
        stack = []
        for i in range(start_index, len(s)):
            if s[i] == '{':
                stack.append(i)
            elif s[i] == '}':
                if stack:
                    start = stack.pop()
                    if not stack:  # 最外层花括号已匹配完成
                        return s[start:i+1]
        return None
    
    # 查找所有可能的JSON开始位置
    for i in range(len(text)):
        if text[i] == '{':
            potential_json = find_balanced_braces(text, i)
            if potential_json:
                try:
                    data = json.loads(potential_json)
                    if "name" in data and "arguments" in data and isinstance(data["arguments"], dict):
                        return data
                except:
                    continue
                    
    return None

In [29]:
# sample_text1 = """
#     这是一些文本，其中包含了一个工具定义：
#     {"name": "calculator", "arguments": {"x": 10, "y": 5, "operation": "add"}}
#     这是一些后续文本
#     """
    
# # 示例2: 包含无效数据的字符串
# sample_text2 = """
# 这是一些文本，但没有有效的工具定义
# {"name": "incomplete"}
# """

# # 示例3: 包含嵌套JSON的复杂例子
# # sample_text3 = """
# # {"name": "data_processor", "arguments": {"config": {"max_items": 100, "filters": ["date", "category"]}, "input_file": "data.csv"}}
# # """
# sample_text3 = """```json
# {"name": "data_processor", "arguments": {"config": {"max_items": 100, "filters": ["date", "category"]}, "input_file": "data.csv"}}```
# """
# for i, sample in enumerate([sample_text1, sample_text2, sample_text3], 1):
#     result = extract_tool_format_from_string(sample)
#     print(result)
#     if result:
#         print(f"示例{i} - 成功提取工具:")
#         print(f"  名称: {result.get('name')}")
#         print(f"  参数: {result.get('arguments')}")
#     else:
#         print(f"示例{i} - 未找到有效的工具定义")
#     print()  

In [34]:
from mcp_client import MCPClientManager

In [60]:
manager=MCPClientManager(server_params_list)

In [59]:
server_params_list = [
    StdioServerParameters(command="npx", args=["--yes", "@modelcontextprotocol/server-filesystem", '/oper/ch/mcp_work']),
    StdioServerParameters(command="uv", args=["run","-m","mcp_server_fetch"]),
    StdioServerParameters(command="npx", args=["-y","@modelcontextprotocol/server-puppeteer"]),
    StdioServerParameters(command="uv",args=["run","/oper/ch/code/chromadb_mcp.py","--ollama_model","bge-large-zh-v1.5:f16","--collection_name","tools_collection"])
]

In [7]:
descriptions,_= await manager.initialize()

In [8]:
result = await manager.run_tool("search_documents",{"query":"write or create new file","include":["documents", "metadatas"],"n_results":3})

In [11]:
response=json.loads(result)

In [47]:
# response['metadatas'][0]

In [13]:
response="\n".join(set([f"{i['tool_name']}:{i['description']}input_spec:{i['input_spec']}" for i in  response['metadatas'][0]]))

In [15]:
await manager.close_all()

In [4]:
import json
from pydantic import BaseModel
from typing import Any,Dict
def get_format_instructions(pydantic_object:BaseModel) -> str:
    """Return the format instructions for the JSON output.

    Returns:
        The format instructions for the JSON output.
    """
    # Copy schema to avoid altering original Pydantic schema.
    schema = dict(pydantic_object.model_json_schema().items())

    # Remove extraneous fields.
    reduced_schema = schema
    if "title" in reduced_schema:
        del reduced_schema["title"]
    if "type" in reduced_schema:
        del reduced_schema["type"]
    # Ensure json in context is well-formed with double quotes.
    schema_str = json.dumps(reduced_schema, ensure_ascii=False)

    return _PYDANTIC_FORMAT_INSTRUCTIONS.format(schema=schema_str)


_PYDANTIC_FORMAT_INSTRUCTIONS = """The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output schema:
```
{schema}
```""" 
class ToolFormat(BaseModel):
    name:str
    arguments:Dict[str,Any]
class CodeWrite(BaseModel):
    codeblock:str
    usage:str

In [14]:
format_prompt=get_format_instructions(ToolFormat)

In [17]:
format_prompt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "arguments": {"additionalProperties": true, "title": "Arguments", "type": "object"}}, "required": ["name", "arguments"]}\n```'

In [ ]:
codeprompt=get_format_instructions(CodeWrite)gte-multilingual-reranker-base/

In [ ]:
import requests

# 替换为你的 Ollama Server 地址（通常是本地的 http://localhost:11434）
OLLAMA_SERVER = "http://localhost:11434"

def chat_with_ollama(model: str, prompt: str):
    url = f"{OLLAMA_SERVER}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # 如果你想流式返回，就设置为 True，并处理响应流
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()  # 抛出 HTTP 错误（如有）

    result = response.json()
    return result["response"]


In [70]:
user_input="""请帮我写一首李白的静夜思，写好后保存文件为`/oper/ch/code/test.txt`"""
prompt = f"""
你可以调用以下工具来完成任务：

{descriptions}

用户的请求是：{user_input}

{format_prompt}
"""

In [14]:
user_input="""请帮我写一首李白的将进酒，写好后保存文件为`/oper/ch/code/test.txt`"""

In [42]:
prompt = f"""
你可以调用以下工具来完成任务：

{descriptions}

用户的请求是：{user_input}

你的任务是：

1. **优先尝试调用可用工具**来完成用户的请求。
2. 如果**没有一个工具可以直接满足**请求，请编写一段 Python 代码，尽可能完成用户的任务。
3. 编写代码时，请确保逻辑清晰、注释完整、可直接运行。
调用工具，输出格式如下：
{format_prompt}
编写代码，输出格式如下：
{codeprompt}
"""

In [4]:
response={
  "name": "write_file",
  "input_dict": {
    "path": "/oper/ch/code/test.txt",
    "content": "床前明月光，\n疑是地上霜。\n举头望明月，\n低头思故乡。"
  }
}

In [20]:
response={
"name": "write_file",
"input_dict": {
"path": "/oper/ch/code/test.txt",
"content": "君不见黄河之水天上来，奔流到海不复回。\n君不见高堂明镜悲白发，朝如青丝暮成雪。\n人生得意须尽欢，莫使金樽空对月。\n天生我材必有用，千金散尽还复来。\n烹羊宰牛且为乐，会须一饮三百杯。\n岑夫子，丹丘生，将进酒，杯莫停。\n与君歌一曲，请君为我倾耳听。\n钟鼓馔玉不足贵，但愿长醉不复醒。\n古来圣贤皆寂寞，惟有饮者留其名。\n陈王昔时宴平乐，斗酒十千恣欢谑。\n主人何为言少钱，径须沽取对君酌。\n五花马，千金裘，呼儿将出换美酒，与尔同销万古愁。"
}
}

In [23]:
result = await manager.run_tool(response['name'],response['input_dict'])

In [22]:
# content=await mcp_tool_run(response['tool'],response['input_dict'],server_params_list,session_index)

In [27]:
# tool_descriptions=''
# for tool in sessions:
#     response=await tool['session'].list_tools()
#     descriptions = "\n".join(f"{line.name}:{line.description}。inputSchema:{line.inputSchema}" for line in response.tools)
#     tool_descriptions+=descriptions

In [30]:
await asyncio.gather(*(close_mcp_session(**e) for e in sessions))

[None, None]

In [75]:
tools=await sessions[0]['session'].list_tools()